In [1]:
# Read chains from e.g. "Eclipse 1919 Stan alpha" notebook, and fit n_components gaussians.
# In fact the posterior is gaussian, so 1 component should be enough.  Then computes the 
# Bayesian evidence analytically from the fit.

from sklearn import mixture
import numpy as np
import matplotlib.pylab as plt 
import scipy.stats as ss 
from datetime import datetime
from scipy.stats import multivariate_normal
import time
from numpy.linalg import inv
import pickle 
from functools import reduce
from scipy import random, linalg
import math
from matplotlib.colors import LogNorm
%matplotlib inline

In [2]:
fileDirectory = './'
samples      = 100000
chainfile    = fileDirectory+'Eclipse_chain_'+str(samples)

p = 4

if(p>2):
    chain = np.loadtxt(chainfile,delimiter=' ',usecols=(2,3,4,5))
else:
    chain = np.loadtxt(chainfile,delimiter=' ',usecols=(2,3))

In [3]:
# GMM = mixture.BayesianGaussianMixture(n_components=1, covariance_type='full', tol=0.001, reg_covar=1e-06, 
#                                   max_iter=100, n_init=1, init_params='kmeans', 
#                                   weight_concentration_prior_type='dirichlet_process', 
#                                   weight_concentration_prior=None, mean_precision_prior=None, 
#                                   mean_prior=None, degrees_of_freedom_prior=None, covariance_prior=None, 
#                                   random_state=None, warm_start=False, verbose=0, verbose_interval=10)

In [4]:
GMM = mixture.BayesianGaussianMixture(n_components=1,covariance_type='full')
GMM.fit(chain)

BayesianGaussianMixture(covariance_prior=None, covariance_type='full',
            degrees_of_freedom_prior=None, init_params='kmeans',
            max_iter=100, mean_precision_prior=None, mean_prior=None,
            n_components=1, n_init=1, random_state=None, reg_covar=1e-06,
            tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
            weight_concentration_prior=None,
            weight_concentration_prior_type='dirichlet_process')

In [5]:
# display predicted scores by the model as a contour plot

if(p==2):
    alphalims = np.linspace(0, 5.)
    alims     = np.linspace(0., 0.3)
#blims     = np.linspace(-0.3, 0.)
#clims     = np.linspace(0.5,1.0)
    alpha, a  = np.meshgrid(alphalims, alims)
    XX        = np.array([alpha.ravel(), a.ravel()]).T
    Z         = -GMM.score_samples(XX)
    Z         = Z.reshape(alpha.shape)

    CS = plt.contour(alpha, a, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                 levels=np.logspace(0,3,10))
    CB = plt.colorbar(CS, shrink=0.8, extend='both')
    plt.scatter(chain[:, 0], chain[:, 1], .8)

    plt.title('Negative log-likelihood predicted by a GMM')
    plt.axis('tight')
    plt.xlabel(r'$\alpha$')
    plt.ylabel('a')
    plt.show()

In [6]:
GMM.means_

array([[ 2.22383838,  0.14438838, -0.10281535,  0.7996291 ]])

In [7]:
GMM.weights_

array([1.])

In [8]:
GMM.covariances_

array([[[ 7.96536275e-01, -3.18392237e-02,  9.63023679e-03,
          4.52129373e-03],
        [-3.18392237e-02,  2.11531121e-03, -7.10457561e-04,
         -3.14637634e-04],
        [ 9.63023679e-03, -7.10457561e-04,  8.33055765e-04,
         -3.45993928e-05],
        [ 4.52129373e-03, -3.14637634e-04, -3.45993928e-05,
          4.37849581e-04]]])

In [9]:
InvC = np.linalg.inv(GMM.covariances_[0])

In [10]:
InvC

array([[   3.17670988,   50.66582733,    6.65788911,    4.13131947],
       [  50.66582733, 1615.81552793,  821.50901231,  702.85553158],
       [   6.65788911,  821.50901231, 1851.78439943,  667.91429769],
       [   4.13131947,  702.85553158,  667.91429769, 2799.07843798]])

In [11]:
D = np.array(p-1)
B = np.array((p-1,1))

D = InvC[1:p,1:p]
B = InvC[1:p,0]
A = InvC[0,0]

InvD = np.linalg.inv(D)

In [12]:
D

array([[1615.81552793,  821.50901231,  702.85553158],
       [ 821.50901231, 1851.78439943,  667.91429769],
       [ 702.85553158,  667.91429769, 2799.07843798]])

In [13]:
B

array([50.66582733,  6.65788911,  4.13131947])

In [14]:
alphabar   = GMM.means_[0,0]
alpha_E    = 1.75
alpha_N    = 0.875
Z_Einstein = 0.5*(alpha_E-alphabar)**2 * (np.dot(B,np.dot(InvD,B.T))-A)
Z_Newton   = 0.5*(alpha_N-alphabar)**2 * (np.dot(B,np.dot(InvD,B.T))-A)

Bayes = np.exp(Z_Einstein-Z_Newton)

In [15]:
print('Bayes factor: %.2f' % Bayes)

Bayes factor: 2.72
